In [16]:
# MJHS vs Vestavia Hills HS vs Homewood HS — ALSDE 2024 (All Students)
""" 
This notebook compares Mortimer Jordan HS (Jefferson County) with Vestavia Hills HS and Homewood HS using publicly available ALSDE Supporting Data (2024).
It assembles key indicators for a fair, apples-to-apples snapshot:

Graduation Rate (%)

College & Career Readiness (%)

Avg Proficiency (ELA/Math/Science) (%) (simple mean across ELA/Math/Science where present)

Chronic Absenteeism (%) (lower is better)

Academic Growth (Index)

Experienced / Novice Educator (%) (lower novice is better)

CCR Cohort Size (Student Count)

Outputs

A comparison CSV and per-metric charts saved to charts/

A one-page scorecard image summarizing all metrics

Reproducibility

Place the ALSDE CSVs in data/raw/Education/ (filenames are pre-mapped below).

If your repo location differs, update BASE.

Run all cells. Charts export to charts/.

Notes & Caveats

Metrics are for All Students where available; proficiency subjects may vary by school/year.

If a metric is missing in one file, the code attempts a reasonable fallback from another ALSDE file.

Column names can change year to year; the notebook uses flexible matching but may need small updates if ALSDE renames headers.

Attribution
Source: ALSDE Supporting Data (2024) — Accountability, CCR/Grad Rate, Educator Experience, Proficiency.
"""

' This notebook compares Mortimer Jordan HS (Jefferson County) with Vestavia Hills HS and Homewood HS using publicly available ALSDE Supporting Data (2024).\nIt assembles key indicators for a fair, apples-to-apples snapshot:\n\nGraduation Rate (%)\n\nCollege & Career Readiness (%)\n\nAvg Proficiency (ELA/Math/Science) (%) (simple mean across ELA/Math/Science where present)\n\nChronic Absenteeism (%) (lower is better)\n\nAcademic Growth (Index)\n\nExperienced / Novice Educator (%) (lower novice is better)\n\nCCR Cohort Size (Student Count)\n\nOutputs\n\nA comparison CSV and per-metric charts saved to charts/\n\nA one-page scorecard image summarizing all metrics\n\nReproducibility\n\nPlace the ALSDE CSVs in data/raw/Education/ (filenames are pre-mapped below).\n\nIf your repo location differs, update BASE.\n\nRun all cells. Charts export to charts/.\n\nNotes & Caveats\n\nMetrics are for All Students where available; proficiency subjects may vary by school/year.\n\nIf a metric is missing 

In [10]:
# ALSDE_SchoolComparison-OTMvsNJ.ipynb — MJHS vs Vestavia HS vs Homewood HS (ALSDE 2024)


from pathlib import Path
from datetime import date
import re, math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from textwrap import fill


# --------------------
# 0) CONFIG
# --------------------
BASE = r"C:/Users/alexmca/Documents/GitHub/KimberlyAL_AnalyticsDataSnapshot"

CCR_CSV  = Path(fr"{BASE}/data/raw/Education/SupportingData_CCRGradRate_2024_000_0000_CCRGradRate - [8_20_2025].csv")
ACCT_CSV = Path(fr"{BASE}/data/raw/Education/SupportingData_Accountability_2024_000_0000_Accountability - [8_20_2025].csv")
PROF_CSV = Path(fr"{BASE}/data/raw/Education/SupportingData_2024_000_0000_Proficiency - [8_20_2025].csv")  # optional
EDU_CSV  = Path(fr"{BASE}/data/raw/Education/SupportingData_Educator_Experienced_2024_000_0000_ExperiencedEducator - [8_20_2025].csv")

OUT_DIR  = Path(fr"{BASE}/charts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# (SYSTEM, OFFICIAL NAME, LABEL SHOWN ON CHARTS)
SCHOOLS = [
    ("Jefferson County",     "Mortimer Jordan High School", "Mortimer Jordan HS"),
    ("Vestavia Hills City",  "Vestavia Hills High School",  "Vestavia Hills HS"),
    ("Homewood City",        "Homewood High School",        "Homewood HS"),
]

# --------------------
# 1) Footers & formatting
# --------------------
TODAY = date.today().isoformat()
SOURCE_ALSDE = "Source: ALSDE Supporting Data (2024): Accountability, CCR/GradRate, Educator Credentials, Proficiency."

fmt_usd = FuncFormatter(lambda y, _: f"${y:,.0f}")
fmt_pct = FuncFormatter(lambda y, _: f"{y:.0f}%")
fmt_num = FuncFormatter(lambda y, _: f"{y:,.0f}")

def save_with_footer(fig, out_path, source_text, note_text=None):
    fig.subplots_adjust(bottom=0.16)
    footer = source_text if not note_text else f"{source_text} • {note_text}"
    fig.text(0.01, 0.02, footer, fontsize=9, ha="left", va="bottom")
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

# --------------------
# 2) Load & helpers
# --------------------
def load_csv(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p, dtype=str, low_memory=False)
    df.columns = [c.strip() for c in df.columns]
    for c in df.columns:
        df[c] = df[c].astype(str).str.strip()
    return df

def pick_col(df: pd.DataFrame, candidates) -> str | None:
    if df is None or df.empty: return None
    for cand in candidates:
        for c in df.columns:
            if c.lower() == cand.lower():
                return c
    for cand in candidates:
        for c in df.columns:
            if cand.lower() in c.lower():
                return c
    return None

def keep_all_groups(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    def is_all(v):
        v = str(v).strip().lower()
        return v == "all" or v.startswith("all ")
    for k in ["Gender","Race","Race/Ethnicity","Ethnicity","Student Group","Sub Population","Subgroup"]:
        if k in out.columns: out = out[out[k].map(is_all)]
    if "Grade" in out.columns:
        g = out["Grade"].str.strip().str.lower()
        if (g == "all grades").any():
            out = out[g == "all grades"]
    return out

def to_float(x):
    if pd.isna(x): return pd.NA
    s = str(x).replace(",", "").replace("%","").strip()
    s = re.sub(r"[^0-9.\-]", "", s)
    try: return float(s)
    except: return pd.NA

def pull_school(df: pd.DataFrame, system: str, school: str) -> pd.DataFrame:
    if df is None or df.empty: return pd.DataFrame()
    sys_col = pick_col(df, ["System","System Name","District","LEA"])
    sch_col = pick_col(df, ["School","School Name"])
    if sys_col is None or sch_col is None: return pd.DataFrame()
    mask = (df[sys_col].str.casefold()==system.casefold()) & (df[sch_col].str.casefold()==school.casefold())
    sub = df[mask]
    if not sub.empty: return sub.copy()
    mask = df[sys_col].str.contains(system, case=False, na=False) & df[sch_col].str.contains(school, case=False, na=False)
    return df[mask].copy()

def acct_score(row_acct: pd.DataFrame, indicator_exact: str):
    if row_acct is None or row_acct.empty: return pd.NA
    ind_col = pick_col(row_acct, ["Indicator"])
    sc_col  = pick_col(row_acct, ["Score"])
    if ind_col is None or sc_col is None: return pd.NA
    rows = row_acct.copy()
    sub_col = pick_col(rows, ["Sub Population","Student Group"])
    if sub_col:
        rows = rows[rows[sub_col].str.strip().str.lower().isin(["all","all students","all subpopulation"])]
    val = rows.loc[rows[ind_col].str.strip().str.lower()==indicator_exact.strip().lower(), sc_col]
    if val.empty: return pd.NA
    return to_float(val.iloc[0])

def ccr_pick_value(row_ccr: pd.DataFrame,
                   col_candidates=("Graduation %","Graduation Rate","Four-Year Graduation Rate","4-Year Graduation Rate"),
                   cohort_candidates=("Cohort","Cohort Type","Cohort Name"),
                   subpop_candidates=("Sub Population","Student Group")):
    if row_ccr is None or row_ccr.empty: return pd.NA
    grad_col = pick_col(row_ccr, list(col_candidates))
    if grad_col is None: return pd.NA
    df = row_ccr.copy()
    sub_col = pick_col(df, list(subpop_candidates))
    if sub_col:
        df_all = df[df[sub_col].str.strip().str.lower().isin(["all","all students","all subpopulation"])]
        if not df_all.empty: df = df_all
    coh_col = pick_col(df, list(cohort_candidates))
    if coh_col:
        pref = df[df[coh_col].str.contains("4", case=False, na=False)]
        if not pref.empty:
            v = to_float(pref[grad_col].iloc[0])
            if pd.notna(v): return v
    for v in df[grad_col]:
        nv = to_float(v)
        if pd.notna(nv): return nv
    return pd.NA

# --------------------
# 3) Load + normalize
# --------------------
ccr  = keep_all_groups(load_csv(CCR_CSV))
acct = keep_all_groups(load_csv(ACCT_CSV))
edu  = keep_all_groups(load_csv(EDU_CSV))
prof = keep_all_groups(load_csv(PROF_CSV)) if PROF_CSV.exists() else pd.DataFrame()

# --------------------
# 4) Metrics per school
# --------------------
def metrics_for(system, school):
    row_ccr  = pull_school(ccr,  system, school)
    row_acct = pull_school(acct, system, school)
    row_prof = pull_school(prof, system, school) if not prof.empty else pd.DataFrame()
    row_edu  = pull_school(edu,  system, school)

    grad = ccr_pick_value(row_ccr)
    if pd.isna(grad):
        grad = acct_score(row_acct, "Graduation Rate")

    ccr_rate = acct_score(row_acct, "College and Career Readiness")
    if pd.isna(ccr_rate):
        ccr_rate = ccr_pick_value(row_ccr, col_candidates=("CCR Attainment %","CCR Rate","CCR %"))

    chronic = acct_score(row_acct, "Chronic Absenteeism")
    growth  = acct_score(row_acct, "Academic Growth")

    exp_col = pick_col(row_edu, ["Experienced Rate","Experienced (%)","Experienced Educators"])
    nov_col = pick_col(row_edu, ["Inexperienced Rate","Inexperienced","Novice","Inexperienced/Novice"])
    exp_pct = to_float(row_edu[exp_col].iloc[0]) if exp_col and not row_edu.empty else pd.NA
    nov_pct = to_float(row_edu[nov_col].iloc[0]) if nov_col and not row_edu.empty else pd.NA

    cohort_col = pick_col(row_ccr, ["Student Count","Students"])
    cohort = to_float(row_ccr[cohort_col].iloc[0]) if cohort_col and not row_ccr.empty else pd.NA

    prof_avg = pd.NA
    if row_prof is not None and not row_prof.empty:
        subj_col = pick_col(row_prof, ["Subject"])
        rate_col = pick_col(row_prof, ["Proficient Rate","% Proficient","Proficiency Rate","Proficiency"])
        if subj_col and rate_col:
            def norm_sub(s):
                s = str(s).lower()
                if "ela" in s or "english language arts" in s: return "ELA"
                if "math" in s: return "Math"
                if "sci" in s: return "Science"
                return None
            best = {}
            for _, r in row_prof.iterrows():
                sub = norm_sub(r[subj_col])
                val = to_float(r[rate_col])
                if sub and pd.notna(val):
                    best[sub] = max(best.get(sub, val), val)
            vals = [v for v in [best.get("ELA"), best.get("Math"), best.get("Science")] if pd.notna(v)]
            if vals: prof_avg = sum(vals)/len(vals)

    return {
        "Graduation Rate (%)": grad,
        "CCR Rate (%)": ccr_rate,
        "Chronic Absenteeism (%)": chronic,
        "Academic Growth (Index)": growth,
        "Experienced Educators (%)": exp_pct,
        "Inexperienced/Novice (%)": nov_pct,
        "CCR Cohort (Student Count)": cohort,
        "Avg Proficiency (ELA/Math/Science) (%)": prof_avg,
    }

rows, debug = [], []
for SYS, SCHOOL, LABEL in SCHOOLS:
    m = metrics_for(SYS, SCHOOL)
    m["System"] = SYS; m["School"] = SCHOOL; m["Label"] = LABEL
    rows.append(m)
    has = {
        "CCR rows": len(pull_school(ccr, SYS, SCHOOL)),
        "ACCT rows": len(pull_school(acct, SYS, SCHOOL)),
        "EDU rows": len(pull_school(edu, SYS, SCHOOL)),
        "PROF rows": len(pull_school(prof, SYS, SCHOOL)) if not prof.empty else 0,
    }
    debug.append((LABEL, has))

compare = pd.DataFrame(rows)[[
    "Label","System","School",
    "Graduation Rate (%)","CCR Rate (%)","Avg Proficiency (ELA/Math/Science) (%)",
    "Chronic Absenteeism (%)","Academic Growth (Index)",
    "Experienced Educators (%)","Inexperienced/Novice (%)",
    "CCR Cohort (Student Count)"
]]

out_csv = OUT_DIR / "alsde_compare_MJHS_Vestavia_Homewood.csv"
compare.to_csv(out_csv, index=False)
print("Wrote:", out_csv)
try:
    from IPython.display import display as _display; _display(compare)
except Exception:
    print(compare)

print("\n--- Debug snapshot ---")
for label, has in debug:
    print(f"[{label}] {has}")
print("-"*60)

# --------------------
# 5) Chart helper
# --------------------
def bar_metric(df, metric, title, ylab, fname, kind="percent", note=None):
    if metric not in df.columns: return
    vals = pd.to_numeric(df[metric], errors="coerce")
    if vals.notna().sum() == 0: return

    fig = plt.figure(figsize=(8.5,5)); ax = fig.gca()
    labels = df["Label"].tolist()
    ax.bar(labels, vals)
    ax.set_title(title); ax.set_xlabel("School"); ax.set_ylabel(ylab)

    if kind == "percent": ax.yaxis.set_major_formatter(fmt_pct)
    elif kind == "number": ax.yaxis.set_major_formatter(fmt_num)
    elif kind == "index":  ax.yaxis.set_major_formatter(fmt_num)

    for i, v in enumerate(vals):
        if pd.notna(v):
            txt = f"{v:.1f}%" if kind=="percent" else f"{v:,.0f}"
            ax.text(i, v, txt, ha="center", va="bottom")

    fig.tight_layout(rect=[0,0.08,1,0.98])
    save_with_footer(fig, OUT_DIR / fname, SOURCE_ALSDE, note_text=note)

# Individual charts
bar_metric(compare, "Graduation Rate (%)",
           "Graduation Rate (ALSDE 2024)", "Percent", "k12_grad_rate.png", kind="percent")

bar_metric(compare, "CCR Rate (%)",
           "College & Career Readiness (ALSDE 2024)", "Percent", "k12_ccr_rate.png", kind="percent")

bar_metric(compare, "Avg Proficiency (ELA/Math/Science) (%)",
           "Avg Proficiency — ELA/Math/Science (ALSDE 2024)", "Percent", "k12_prof_avg.png", kind="percent",
           note="Simple mean across ELA/Math/Science where present.")

bar_metric(compare, "Chronic Absenteeism (%)",
           "Chronic Absenteeism (ALSDE 2024) — lower is better", "Percent", "k12_chronic_abs.png", kind="percent")

bar_metric(compare, "Academic Growth (Index)",
           "Academic Growth (ALSDE 2024)", "Index", "k12_growth.png", kind="index")

bar_metric(compare, "Experienced Educators (%)",
           "Experienced Educators (ALSDE 2024)", "Percent", "k12_experienced.png", kind="percent")

bar_metric(compare, "Inexperienced/Novice (%)",
           "Novice/Inexperienced (ALSDE 2024) — lower is better", "Percent", "k12_novice.png", kind="percent")

bar_metric(compare, "CCR Cohort (Student Count)",
           "CCR Cohort Size (CCR file)", "Students", "k12_ccr_cohort.png", kind="number")

# --------------------
# 6) Scorecard
# --------------------
def export_scorecard(df: pd.DataFrame, outfile, title=None, school_order=None,
                     wrap_width=14, tick_fontsize=9, tick_rotation=10):
    """
    Generate a clean multi-panel scorecard with wrapped tick labels and extra headroom.
    """
    if school_order:
        df = df.set_index("Label").loc[school_order].reset_index()

    labels = df["Label"].tolist()
    # Wrap labels to avoid overlap (e.g., "Mortimer\nJordan HS")
    wrapped_labels = [fill(l, width=wrap_width) for l in labels]

    # Panels (only keep those that exist & have data)
    panels = [
        ("Graduation Rate (%)",                    "Graduation Rate",                        "Percent",  "percent"),
        ("CCR Rate (%)",                           "College & Career Readiness",             "Percent",  "percent"),
        ("Avg Proficiency (ELA/Math/Science) (%)","Avg Proficiency (ELA/Math/Sci)",         "Percent",  "percent"),
        ("Chronic Absenteeism (%)",                "Chronic Absenteeism (lower is better)",  "Percent",  "percent"),
        ("Experienced Educators (%)",              "Experienced Educators",                  "Percent",  "percent"),
        ("Inexperienced/Novice (%)",               "Novice/Inexperienced (lower is better)", "Percent",  "percent"),
        ("Academic Growth (Index)",                "Academic Growth (Index)",                "Index",    "index"),
        ("CCR Cohort (Student Count)",             "CCR Cohort Size",                        "Students", "number"),
    ]
    panels = [p for p in panels
              if p[0] in df.columns and pd.to_numeric(df[p[0]], errors="coerce").notna().any()]
    if not panels:
        print("No metrics available for scorecard."); return

    # Layout
    n = len(panels)
    ncols = 4 if n >= 4 else n
    nrows = int((n + ncols - 1) // ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(4.8*ncols, 4.1*nrows))
    if nrows == 1 and ncols == 1:
        axes = [[axes]]
    elif nrows == 1:
        axes = [axes]
    elif ncols == 1:
        axes = [[ax] for ax in axes]

    def set_ylim_with_headroom(ax, vals, kind):
        # Create some headroom for value labels
        vals_clean = [v for v in vals if pd.notna(v)]
        if not vals_clean:
            return
        top = max(vals_clean)
        if kind == "percent":
            # keep under/around 100 but allow a bit of space
            top = 105 if top >= 95 else top * 1.12
        elif kind in ("index", "number"):
            top = top * 1.15
        ax.set_ylim(0, top)

    for i, (col, panel_title, ylab, kind) in enumerate(panels):
        r, c = divmod(i, ncols)
        ax = axes[r][c]
        vals = [pd.to_numeric(x, errors="coerce") for x in df[col].tolist()]
        x = list(range(len(labels)))
        ax.bar(x, vals)
        ax.set_title(panel_title, fontsize=12)
        ax.set_ylabel(ylab)
        ax.set_xticks(x)
        ax.set_xticklabels(wrapped_labels, fontsize=tick_fontsize, rotation=tick_rotation, ha="right")

        # y-axis formatting
        if kind == "percent":
            ax.yaxis.set_major_formatter(fmt_pct)
        elif kind == "number":
            ax.yaxis.set_major_formatter(fmt_num)
        elif kind == "index":
            ax.yaxis.set_major_formatter(fmt_num)

        set_ylim_with_headroom(ax, vals, kind)

        # Annotate values
        y0, y1 = ax.get_ylim()
        bump = (y1 - y0) * 0.03
        for j, v in enumerate(vals):
            if pd.notna(v):
                txt = f"{v:.1f}%" if kind == "percent" else f"{v:,.0f}"
                ax.text(j, v + bump, txt, ha="center", va="bottom", fontsize=10)

    # Remove unused axes
    total_axes = nrows * ncols
    if n < total_axes:
        for k in range(n, total_axes):
            r, c = divmod(k, ncols)
            fig.delaxes(axes[r][c])

    if title is None:
        title = "High School Snapshot — ALSDE 2024 (All Students)"
    fig.suptitle(title, fontsize=16)

    # Source / notes footer
    fig.text(
        0.01, 0.01,
        "Source: ALSDE Supporting Data (2024). Notes: lower is better for chronic absenteeism and novice %. "
        "Avg Proficiency is a simple mean across ELA/Math/Science where present.",
        fontsize=9, ha="left", va="bottom"
    )

    fig.tight_layout(rect=[0, 0.05, 1, 0.92])
    fig.savefig(outfile, dpi=200)
    plt.close(fig)
    print("Scorecard saved to:", outfile)

# Call it (unchanged):
export_scorecard(
    compare,
    OUT_DIR / "k12_scorecard_MJHS_Vestavia_Homewood.png",
    title="Mortimer Jordan HS vs Vestavia Hills HS vs Homewood HS — ALSDE 2024 (All Students)",
    school_order=[s[2] for s in SCHOOLS]
)

Wrote: C:\Users\alexmca\Documents\GitHub\KimberlyAL_AnalyticsDataSnapshot\charts\alsde_compare_MJHS_Vestavia_Homewood.csv


,Label,System,School,Graduation Rate (%),CCR Rate (%),Avg Proficiency (ELA/Math/Science) (%),Chronic Absenteeism (%),Academic Growth (Index),Experienced Educators (%),Inexperienced/Novice (%),CCR Cohort (Student Count)
0,Mortimer Jordan HS,Jefferson County,Mortimer Jordan High School,94.39,96.94,30.753333,23.81,92.56,92.30,7.70,196.0
1,Vestavia Hills HS,Vestavia Hills City,Vestavia Hills High School,99.03,94.75,68.436667,10.06,99.86,96.46,3.54,514.0
2,Homewood HS,Homewood City,Homewood High School,93.36,95.35,65.796667,8.48,100.00,96.01,3.99,301.0



--- Debug snapshot ---
[Mortimer Jordan HS] {'CCR rows': 1, 'ACCT rows': 6, 'EDU rows': 1, 'PROF rows': 3}
[Vestavia Hills HS] {'CCR rows': 1, 'ACCT rows': 6, 'EDU rows': 1, 'PROF rows': 3}
[Homewood HS] {'CCR rows': 1, 'ACCT rows': 6, 'EDU rows': 1, 'PROF rows': 3}
------------------------------------------------------------
Scorecard saved to: C:\Users\alexmca\Documents\GitHub\KimberlyAL_AnalyticsDataSnapshot\charts\k12_scorecard_MJHS_Vestavia_Homewood.png
